# Exploratory Data Analysis

This notebook aims to explore the car feature dataset and uncover trends to help predict car
prices. This notebookk will:

- rank utility of features using mutual infomation
- explore relationship of variables with price

In [ ]:
from eda import *
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()

In [ ]:
df = pd.read_pickle('car_data.pkl')
df.head()

In [ ]:
X = df.copy()
y = X.pop('price')

## Get mutual infomation scores

In [ ]:
mi_scores = make_mi_scores(X, y)
mi_scores[::3]

Key variables like manufactured year and mileage look they have a effect of price as expected.
By this metric valiables like doors and seats have little to no effect on price.

In [ ]:
plot_mi_scores(mi_scores)

The price distribution shows that prices range up to £200,000 on the extreme upper end. However morst of the price data is below £60,000 as seen on the right side of the figure below. The data is also right skewed.

In [ ]:
plot_price_historgam_subplot(df)

Plotting some of the numerical variables against price we can see trends below.
1. Car length is ranked 1 for the mutual information scores and as seen in the chart there is a positive correlation between car length and price.
2. Price seems to increase with boot space, but more so with the seats down measurement. This is also confirmed by the mutual information scores as the seats down measurement was ranked higher than the seats up measurement.
3. Price increases with the wheelbase. There are some incorrect figures as some wheelbase data points are at 0mm which makes no sense.
4. Height, manufactured ear and engine power all have strong positive correlations with price. 

In [ ]:
plot_eda_subplot1(df)

In [ ]:
df_sample = df.sample(frac=.4, random_state=1)
plot_eda_subplot1(df_sample)

1. Engine torque, mileage, fuel tank capacity and CO2 emmissions seem to have strong correlations with price and will likely be useful in price prediction.
2. The relationship between annual tax and price is unclear. Tax is ranked low on the mutual information scores.
3. Price decreases with an increase in number of owners. Number of owners could be closely correlated with mileage.

In [ ]:
fig, ax = plt.subplots(2, 3, figsize=[24, 15])

# fig 1
ax[0,0].scatter(df_sample.engine_torque, df_sample.price, alpha=0.5)
ax[0,0].set(ylabel= 'Price (£)', xlabel='Engine Torque (lbs/ft)')

# fig 2
ax[0,1].scatter(df_sample.mileage, df_sample.price, alpha=0.5)
ax[0,1].set(ylabel= 'Price (£)', xlabel='Mileage (miles)')

# fig 3
ax[0,2].scatter(df_sample.fuel_tank_capacity, df_sample.price, alpha=0.5)
ax[0,2].set(ylabel= 'Price (£)', xlabel='Fuel Tank Capacity (L)')

# fig 4
ax[1,0].scatter(df_sample.co2_emissions, df_sample.price, alpha=0.5)
ax[1,0].set(ylabel= 'Price (£)', xlabel='CO2 Emmissions (g/km)')

# fig 5
ax[1,1].scatter(df_sample.tax, df_sample.price, alpha=0.5)
ax[1,1].set(ylabel= 'Price (£)', xlabel='Annual Tax (£)')

# fig 6
ax[1,2].scatter(df_sample.number_of_owners, df_sample.price, alpha=0.5)
ax[1,2].set(ylabel= 'Price (£)', xlabel='Number of Owners')

1. Price increases with top speed and engine size.
2. Total sellers reviews does not have a clear trend with price.
3. Combined, urban and extra urban ratings all seem to have a slight negative correlation with price although it's not very clear. These variables may not be good predictors of price.

In [ ]:
fig, ax = plt.subplots(2, 3, figsize=[24, 15])

# fig 1
ax[0,0].scatter(df_sample.top_speed, df_sample.price, alpha=0.5)
ax[0,0].set(ylabel= 'Price (£)', xlabel='Top Speed (mph)')

# fig 2
ax[0,1].scatter(df_sample.total_reviews, df_sample.price, alpha=0.5)
ax[0,1].set(ylabel= 'Price (£)', xlabel='Total Reviews')

# fig 3
ax[0,2].scatter(df_sample.engine_size, df_sample.price, alpha=0.5)
ax[0,2].set(ylabel= 'Price (£)', xlabel='Engine Size (L)')

# fig 4
ax[1,0].scatter(df_sample.combined, df_sample.price, alpha=0.5)
ax[1,0].set(ylabel= 'Price (£)', xlabel='Combined (mpg)')

# fig 5
ax[1,1].scatter(df_sample.urban, df_sample.price, alpha=0.5)
ax[1,1].set(ylabel= 'Price (£)', xlabel='Urban (mpg)')

# fig 6
ax[1,2].scatter(df_sample.extra_urban, df_sample.price, alpha=0.5)
ax[1,2].set(ylabel= 'Price (£)', xlabel='Extra Urban (mpg)')

1. Longitude, doors, seats and number of photos do not have a clear correlation with price so will probably be useless for price prediction.
2. Seller ratings is positively correlated with price. However not all sellers have ratings as some are private. This can lead a lot of data loss if this variable is used.
3. Price increases with number of valves and cylinders.

In [ ]:
fig, ax = plt.subplots(2, 3, figsize=[24, 15])

# fig 1
ax[0,0].scatter(df_sample.valves, df_sample.price, alpha=0.5)
ax[0,0].set(ylabel= 'Price (£)', xlabel='Valves')

# fig 2
ax[0,1].scatter(df_sample.vehicle_location_longitude, df_sample.price, alpha=0.5)
ax[0,1].set(ylabel= 'Price (£)', xlabel='Vehicle Location Longitude')

# fig 3
ax[0,2].scatter(df_sample.cylinders, df_sample.price, alpha=0.5)
ax[0,2].set(ylabel= 'Price (£)', xlabel='Cylinders')

# fig 4
ax[1,0].scatter(df_sample.seller_rating, df_sample.price, alpha=0.5)
ax[1,0].set(ylabel= 'Price (£)', xlabel='Seller Rating')

# fig 5
ax[1,1].scatter(df_sample.number_of_photos, df_sample.price, alpha=0.5)
ax[1,1].set(ylabel= 'Price (£)', xlabel='Number of Photos ')

# fig 6
ax[1,2].scatter(df_sample.doors, df_sample.price, alpha=0.5)
ax[1,2].set(ylabel= 'Price (£)', xlabel='Doors')

In [ ]:
fig, ax = plt.subplots(figsize=[8, 5])

ax.scatter(df_sample.seats, df_sample.price, alpha=0.1)
ax.set(ylabel= 'Price (£)', xlabel='Seats')


1. Figures 1 and 2 show the top ten makes and models in terms of price. It includes luxury cars such as maseratia and porshe.
2. Automatic transmission cars have a higher median price than manuals
3. Fuel type seems to have an effect on price. Diesel plugin hybrids have the highest median price.
4. Body type does not seem to have much effect on price with the exception of the van. This has a higher median than the other body types.

In [ ]:
mean_price_by_make = df.groupby('make')['price'].mean().round(0).dropna().astype('int').sort_values(ascending=False)
mean_price_by_model = df.groupby('model')['price'].mean().round(0).dropna().astype('int').sort_values(ascending=False)

In [ ]:
df.fuel_type = df.fuel_type.astype('object').astype('category')
df.body_type = df.body_type.astype('object').astype('category')

fig, ax = plt.subplots(2, 3, figsize=[24, 15])
base_color = sns.color_palette()[0]

N = 10
ticks = np.arange(N) 
width = 0.7   

# fig 1 
ax[0,0].bar(ticks, mean_price_by_make.values[:N], width)
ax[0,0].set_xticks(ticks) 
ax[0,0].set_xticklabels(mean_price_by_make.index[:N], rotation=45)
ax[0,0].set(ylabel= 'Mean Price (£)', xlabel='Car Make')

# fig 2
ax[0,1].bar(ticks, mean_price_by_model.values[:N], width)
ax[0,1].set_xticks(ticks) 
ax[0,1].set_xticklabels(mean_price_by_model.index[:N], rotation=45)
ax[0,1].set(ylabel= 'Mean Price (£)', xlabel='Model')

# fig 3
sns.boxplot(data = df, x = 'transmission', y = 'price', ax=ax[0,2], color=base_color)
ax[0,2].set(ylabel= 'Price (£)', xlabel='Transmission')

# fig 4
sns.boxplot(data = df, x = 'fuel_type', y = 'price', ax=ax[1,0], color=base_color)
ax[1,0].tick_params(axis='x', labelrotation=45)
ax[1,0].set(ylabel= 'Price (£)', xlabel='Fuel Type')

# fig 5
sns.boxplot(data = df, x = 'body_type', y = 'price', ax=ax[1,1], color=base_color)
ax[1,1].tick_params(axis='x', labelrotation=45)
ax[1,1].set(ylabel= 'Price (£)', xlabel='Body Type')

fig.tight_layout()
fig.delaxes(ax[1,2])